Import everything needed to grab files from google drive and use them.

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

Setup up drive with authentication

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Get training and testing files from Google drive.

In [0]:
fid = drive.ListFile({'q':"title='input.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('input.zip')

f.keys()

!unzip input.zip

PATH = '/content/input'
os.listdir(PATH)
train_dir = os.path.join(PATH, 'Train')
test_dir = os.path.join(PATH, 'Test')

train_fire_dir = os.path.join(train_dir, 'fire')
train_no_fire_dir = os.path.join(train_dir, 'no_fire')
test_fire_dir = os.path.join(test_dir, 'fire')
test_no_fire_dir = os.path.join(test_dir, 'no_fire')

print(len(os.listdir(train_fire_dir)))
print(len(os.listdir(train_no_fire_dir)))
print(len(os.listdir(test_fire_dir)))
print(len(os.listdir(test_no_fire_dir)))


Use existing weights from previously developed models.

In [0]:
import tensorflow as tf
tf.__version__
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
#resnet_weights_path = 'input/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
resnet_weights_path = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.2/'
                       'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

Train the model based on the fire images recieved from the drive.

In [22]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224
batch_size = 20

num_fire_train = len(os.listdir(train_fire_dir))
num_no_fire_train = len(os.listdir(train_no_fire_dir))
num_fire_test = len(os.listdir(test_fire_dir))
num_no_fire_test = len(os.listdir(test_no_fire_dir))

total_train = num_fire_train + num_no_fire_train
total_test = num_fire_test + num_no_fire_test

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory(
        directory=train_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        directory=test_dir,
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit(
        train_generator,
        steps_per_epoch=12,
        validation_data=validation_generator,
        validation_steps=3)

Found 649 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
12/12 [==============================] - 45s 4s/step - loss: 0.0979 - accuracy: 0.9607 - val_loss: 0.1939 - val_accuracy: 0.9167
